# Solutions: Odibi Configuration System Exercises

## Solution 1: PartitionStrategy Enum

In [ ]:
from enum import Enum
from typing import List
from pydantic import BaseModel, Field, ValidationError

class PartitionStrategy(str, Enum):
    """Partition strategies for data writing."""
    NONE = "none"
    HASH = "hash"
    RANGE = "range"
    DATE = "date"

class PartitionConfig(BaseModel):
    strategy: PartitionStrategy
    columns: List[str] = Field(default_factory=list)

# Test valid
config1 = PartitionConfig(strategy="hash", columns=["user_id"])
print(f"✅ Valid: {config1}")

config2 = PartitionConfig(strategy=PartitionStrategy.DATE, columns=["created_date"])
print(f"✅ Valid: {config2}")

# Test invalid
try:
    config3 = PartitionConfig(strategy="random", columns=[])
except ValidationError as e:
    print(f"\n❌ Invalid strategy: {e}")

## Solution 2: Enhanced WriteConfig with Partitioning

In [ ]:
from typing import Optional, List, Dict, Any
from pydantic import BaseModel, Field, model_validator

class WriteMode(str, Enum):
    OVERWRITE = "overwrite"
    APPEND = "append"

class WriteConfig(BaseModel):
    """Configuration for writing data."""
    connection: str
    format: str
    table: Optional[str] = None
    path: Optional[str] = None
    mode: WriteMode = WriteMode.OVERWRITE
    options: Dict[str, Any] = Field(default_factory=dict)
    
    # Partitioning
    partition_strategy: PartitionStrategy = PartitionStrategy.NONE
    partition_columns: List[str] = Field(default_factory=list)
    
    @model_validator(mode="after")
    def check_table_or_path(self):
        """Ensure either table or path is provided."""
        if not self.table and not self.path:
            raise ValueError("Either 'table' or 'path' must be provided")
        return self
    
    @model_validator(mode="after")
    def check_partition_columns(self):
        """If partitioning is enabled, columns must be specified."""
        if self.partition_strategy != PartitionStrategy.NONE:
            if not self.partition_columns:
                raise ValueError(
                    f"Partition strategy '{self.partition_strategy}' requires partition_columns. "
                    f"Specify at least one column to partition by."
                )
        return self

# Test 1: No partitioning (valid)
write1 = WriteConfig(
    connection="local",
    format="parquet",
    path="output/data.parquet"
)
print(f"✅ No partitioning: {write1.partition_strategy}\n")

# Test 2: With partitioning (valid)
write2 = WriteConfig(
    connection="delta",
    format="delta",
    table="sales",
    partition_strategy="date",
    partition_columns=["created_date", "region"]
)
print(f"✅ With partitioning: {write2.partition_strategy}, columns={write2.partition_columns}\n")

# Test 3: Partitioning without columns (invalid)
try:
    write3 = WriteConfig(
        connection="delta",
        format="delta",
        table="sales",
        partition_strategy="hash"
        # Missing partition_columns!
    )
except ValidationError as e:
    print(f"❌ Partitioning without columns:\n{e}")

## Solution 3: ScheduleConfig

In [ ]:
class ScheduleType(str, Enum):
    """Pipeline schedule types."""
    MANUAL = "manual"
    CRON = "cron"
    INTERVAL = "interval"

class ScheduleConfig(BaseModel):
    """Configuration for pipeline scheduling."""
    type: ScheduleType
    cron_expression: Optional[str] = Field(
        default=None,
        description="Cron expression (required if type is CRON)"
    )
    interval_minutes: Optional[int] = Field(
        default=None,
        ge=1,
        description="Interval in minutes (required if type is INTERVAL)"
    )
    
    @model_validator(mode="after")
    def check_schedule_requirements(self):
        """Validate schedule type-specific requirements."""
        if self.type == ScheduleType.CRON:
            if not self.cron_expression:
                raise ValueError(
                    "Schedule type 'cron' requires 'cron_expression'. "
                    "Example: '0 0 * * *' for daily at midnight."
                )
        
        if self.type == ScheduleType.INTERVAL:
            if not self.interval_minutes:
                raise ValueError(
                    "Schedule type 'interval' requires 'interval_minutes'. "
                    "Specify how often the pipeline should run (minimum 1 minute)."
                )
        
        return self

# Test 1: Manual schedule
schedule1 = ScheduleConfig(type="manual")
print(f"✅ Manual schedule: {schedule1}\n")

# Test 2: Cron schedule
schedule2 = ScheduleConfig(
    type="cron",
    cron_expression="0 0 * * *"  # Daily at midnight
)
print(f"✅ Cron schedule: {schedule2.cron_expression}\n")

# Test 3: Interval schedule
schedule3 = ScheduleConfig(
    type="interval",
    interval_minutes=30
)
print(f"✅ Interval schedule: every {schedule3.interval_minutes} minutes\n")

# Test 4: Cron without expression (invalid)
try:
    schedule4 = ScheduleConfig(type="cron")
except ValidationError as e:
    print(f"❌ Cron without expression:\n{e}\n")

# Test 5: Interval without minutes (invalid)
try:
    schedule5 = ScheduleConfig(type="interval")
except ValidationError as e:
    print(f"❌ Interval without minutes:\n{e}")

## Solution 4: Storage Configs - Discriminated Union

In [ ]:
from typing import Union

class StorageType(str, Enum):
    """Supported storage types."""
    S3 = "s3"
    GCS = "gcs"
    AZURE = "azure"
    LOCAL = "local"

class BaseStorageConfig(BaseModel):
    """Base configuration for all storage types."""
    type: StorageType
    encryption: bool = False

class S3StorageConfig(BaseStorageConfig):
    """AWS S3 storage configuration."""
    type: StorageType = StorageType.S3
    bucket: str
    region: str
    access_key_id: Optional[str] = None  # Can use IAM role instead

class GCSStorageConfig(BaseStorageConfig):
    """Google Cloud Storage configuration."""
    type: StorageType = StorageType.GCS
    bucket: str
    project: str

class AzureStorageConfig(BaseStorageConfig):
    """Azure Blob Storage configuration."""
    type: StorageType = StorageType.AZURE
    account_name: str
    container: str
    sas_token: Optional[str] = None  # Can use managed identity instead

class LocalStorageConfig(BaseStorageConfig):
    """Local filesystem storage configuration."""
    type: StorageType = StorageType.LOCAL
    base_path: str

# Union type for polymorphic storage configs
StorageConfig = Union[
    S3StorageConfig,
    GCSStorageConfig,
    AzureStorageConfig,
    LocalStorageConfig,
]

# Test S3
s3 = S3StorageConfig(
    bucket="my-data-bucket",
    region="us-east-1",
    encryption=True
)
print(f"✅ S3: {s3}\n")

# Test GCS
gcs = GCSStorageConfig(
    bucket="my-gcs-bucket",
    project="my-project-123"
)
print(f"✅ GCS: {gcs}\n")

# Test Azure
azure = AzureStorageConfig(
    account_name="myaccount",
    container="data",
    sas_token="sv=2021..."
)
print(f"✅ Azure: {azure}\n")

# Test Local
local = LocalStorageConfig(
    base_path="/data/warehouse"
)
print(f"✅ Local: {local}")

## Solution 5: Field Validator - Cron Expression

In [ ]:
from pydantic import field_validator
import re

class ScheduleConfig(BaseModel):
    """Configuration for pipeline scheduling with cron validation."""
    type: ScheduleType
    cron_expression: Optional[str] = None
    interval_minutes: Optional[int] = None
    
    @field_validator("cron_expression")
    @classmethod
    def validate_cron(cls, v):
        """Validate cron expression format."""
        if v is None:
            return v
        
        # Split into parts
        parts = v.split()
        
        # Must have exactly 5 parts
        if len(parts) != 5:
            raise ValueError(
                f"Cron expression must have exactly 5 parts (minute hour day month weekday). "
                f"Got {len(parts)} parts: '{v}'. "
                f"Example: '0 0 * * *' for daily at midnight."
            )
        
        # Each part should match cron syntax
        # Simple validation: *, number, range (1-5), step (*/5), or list (1,2,3)
        cron_pattern = r'^(\*|\d+|\d+-\d+|\*/\d+|\d+(,\d+)*)$'
        
        for i, part in enumerate(parts):
            if not re.match(cron_pattern, part):
                field_names = ["minute", "hour", "day", "month", "weekday"]
                raise ValueError(
                    f"Invalid cron {field_names[i]} field: '{part}'. "
                    f"Must be *, a number, a range (1-5), a step (*/5), or a list (1,2,3)."
                )
        
        return v
    
    @model_validator(mode="after")
    def check_schedule_requirements(self):
        """Validate schedule type-specific requirements."""
        if self.type == ScheduleType.CRON and not self.cron_expression:
            raise ValueError("Schedule type 'cron' requires 'cron_expression'")
        if self.type == ScheduleType.INTERVAL and not self.interval_minutes:
            raise ValueError("Schedule type 'interval' requires 'interval_minutes'")
        return self

# Test valid cron expressions
valid_crons = [
    "0 0 * * *",      # Daily at midnight
    "*/15 * * * *",   # Every 15 minutes
    "0 9-17 * * 1-5", # Hourly, 9am-5pm, weekdays
    "30 2 * * 0",     # Weekly on Sunday at 2:30am
    "0 0 1 * *",      # Monthly on 1st at midnight
]

for cron in valid_crons:
    schedule = ScheduleConfig(type="cron", cron_expression=cron)
    print(f"✅ Valid cron: {cron}")

# Test invalid cron expressions
invalid_crons = [
    "0 0 * *",           # Too few parts
    "0 0 * * * *",       # Too many parts
    "abc 0 * * *",       # Invalid character
    "0 0-25 * * *",      # Invalid hour range
]

print("\nTesting invalid cron expressions:\n")
for cron in invalid_crons:
    try:
        schedule = ScheduleConfig(type="cron", cron_expression=cron)
    except ValidationError as e:
        print(f"❌ '{cron}': {e.errors()[0]['msg']}\n")

## Solution 6: Complete Pipeline Config

In [ ]:
import yaml

# First, define all necessary classes (simplified versions)
class ReadConfig(BaseModel):
    connection: str
    format: str
    table: Optional[str] = None
    path: Optional[str] = None

class TransformConfig(BaseModel):
    steps: List[str]

class NodeConfig(BaseModel):
    name: str
    description: Optional[str] = None
    depends_on: List[str] = Field(default_factory=list)
    read: Optional[ReadConfig] = None
    transform: Optional[TransformConfig] = None
    write: Optional[WriteConfig] = None

class PipelineConfig(BaseModel):
    """Enhanced pipeline configuration with scheduling."""
    pipeline: str
    description: Optional[str] = None
    layer: Optional[str] = None
    schedule: Optional[ScheduleConfig] = None  # New!
    nodes: List[NodeConfig]

# YAML configuration
pipeline_yaml = """
pipeline: daily_sales_processing
description: Process daily sales data with partitioning
layer: silver

schedule:
  type: cron
  cron_expression: "0 2 * * *"  # Daily at 2am

nodes:
  - name: read_sales
    description: Read raw sales data from Delta
    read:
      connection: delta
      format: delta
      table: raw.sales
  
  - name: clean_sales
    description: Clean and validate sales data
    depends_on: [read_sales]
    transform:
      steps:
        - SELECT * FROM read_sales WHERE amount > 0
        - SELECT * FROM data WHERE customer_id IS NOT NULL
  
  - name: aggregate_by_date
    description: Aggregate sales by date and region
    depends_on: [clean_sales]
    transform:
      steps:
        - |
          SELECT 
            DATE(created_at) as sale_date,
            region,
            COUNT(*) as total_orders,
            SUM(amount) as total_revenue
          FROM clean_sales
          GROUP BY DATE(created_at), region
    write:
      connection: delta
      format: delta
      table: silver.sales_daily
      mode: overwrite
      partition_strategy: date
      partition_columns: [sale_date, region]
"""

# Load and validate
config = PipelineConfig(**yaml.safe_load(pipeline_yaml))

# Print summary
print("="*60)
print(f"Pipeline: {config.pipeline}")
print(f"Description: {config.description}")
print(f"Layer: {config.layer}")
print(f"\nSchedule: {config.schedule.type}")
if config.schedule.cron_expression:
    print(f"  Cron: {config.schedule.cron_expression}")

print(f"\nNodes ({len(config.nodes)}):")
for node in config.nodes:
    print(f"\n  📦 {node.name}")
    if node.description:
        print(f"     {node.description}")
    if node.depends_on:
        print(f"     Depends on: {', '.join(node.depends_on)}")
    if node.read:
        print(f"     Read: {node.read.format} from {node.read.table or node.read.path}")
    if node.transform:
        print(f"     Transform: {len(node.transform.steps)} steps")
    if node.write:
        print(f"     Write: {node.write.format} to {node.write.table or node.write.path}")
        if node.write.partition_strategy != PartitionStrategy.NONE:
            print(f"     Partitioned by: {node.write.partition_strategy} on {node.write.partition_columns}")

print("="*60)

## Solution 7: Error Message Quality

In [ ]:
class DataPipelineConfig(BaseModel):
    """Example config with high-quality error messages."""
    name: str
    source_connection: str
    target_connection: str
    available_connections: List[str] = Field(exclude=True)  # Not in YAML, set programmatically
    partitions: int = Field(ge=1, le=1000)
    
    @model_validator(mode="after")
    def validate_connections(self):
        """Validate connections exist and are different."""
        # Check source exists
        if self.available_connections and self.source_connection not in self.available_connections:
            raise ValueError(
                f"Source connection '{self.source_connection}' not found. "
                f"Available connections: {', '.join(self.available_connections)}. "
                f"Add it to the 'connections' section of your config or fix the typo."
            )
        
        # Check target exists
        if self.available_connections and self.target_connection not in self.available_connections:
            raise ValueError(
                f"Target connection '{self.target_connection}' not found. "
                f"Available connections: {', '.join(self.available_connections)}. "
                f"Add it to the 'connections' section of your config or fix the typo."
            )
        
        # Check they're different
        if self.source_connection == self.target_connection:
            raise ValueError(
                f"Source and target connections must be different (both are '{self.source_connection}'). "
                f"Reading and writing to the same connection can cause data corruption. "
                f"Use different connections or add logic to prevent overwrites."
            )
        
        return self

# Test error scenarios
available = ["bronze", "silver", "gold"]

print("Testing error messages:\n")

# Error 1: Source connection not found
try:
    config = DataPipelineConfig(
        name="test",
        source_connection="raw",  # Doesn't exist
        target_connection="silver",
        partitions=10,
        available_connections=available
    )
except ValidationError as e:
    print("❌ Error 1 - Source not found:")
    print(f"{e.errors()[0]['msg']}\n")

# Error 2: Same source and target
try:
    config = DataPipelineConfig(
        name="test",
        source_connection="silver",
        target_connection="silver",  # Same!
        partitions=10,
        available_connections=available
    )
except ValidationError as e:
    print("❌ Error 2 - Same connection:")
    print(f"{e.errors()[0]['msg']}\n")

# Error 3: Partitions out of range
try:
    config = DataPipelineConfig(
        name="test",
        source_connection="bronze",
        target_connection="silver",
        partitions=5000,  # Too many!
        available_connections=available
    )
except ValidationError as e:
    print("❌ Error 3 - Too many partitions:")
    print(f"{e.errors()[0]['msg']}\n")

## Solution 8: Config Inheritance

In [ ]:
from typing import Dict, Any
from copy import deepcopy

def merge_configs(base: Dict[str, Any], override: Dict[str, Any]) -> Dict[str, Any]:
    """Merge two config dictionaries, override takes precedence.
    
    Handles nested dictionaries recursively.
    """
    result = deepcopy(base)
    
    for key, value in override.items():
        if key in result and isinstance(result[key], dict) and isinstance(value, dict):
            # Recursively merge nested dicts
            result[key] = merge_configs(result[key], value)
        else:
            # Override takes precedence
            result[key] = deepcopy(value)
    
    return result

# Test
base_config = {
    "project": "data_pipeline",
    "logging": {
        "level": "INFO",
        "structured": False,
        "metadata": {
            "team": "data-eng",
            "environment": "dev"
        }
    },
    "retry": {
        "enabled": True,
        "max_attempts": 3
    },
    "connections": {
        "local": {
            "type": "local",
            "base_path": "./data"
        }
    }
}

# Production overrides
prod_override = {
    "logging": {
        "level": "ERROR",
        "structured": True,
        "metadata": {
            "environment": "production"  # Only override environment, keep team
        }
    },
    "retry": {
        "max_attempts": 5  # More retries in prod
    },
    "connections": {
        "delta": {  # Add new connection
            "type": "delta",
            "catalog": "prod"
        }
    }
}

merged = merge_configs(base_config, prod_override)

print("Base Config:")
print(yaml.dump(base_config, default_flow_style=False))

print("\nProduction Override:")
print(yaml.dump(prod_override, default_flow_style=False))

print("\nMerged Config:")
print(yaml.dump(merged, default_flow_style=False))

# Verify specific merges
assert merged["logging"]["level"] == "ERROR"  # Overridden
assert merged["logging"]["structured"] == True  # Overridden
assert merged["logging"]["metadata"]["team"] == "data-eng"  # Kept from base
assert merged["logging"]["metadata"]["environment"] == "production"  # Overridden
assert merged["retry"]["enabled"] == True  # Kept from base
assert merged["retry"]["max_attempts"] == 5  # Overridden
assert "local" in merged["connections"]  # Kept from base
assert "delta" in merged["connections"]  # Added from override

print("\n✅ All assertions passed!")

## Solution 9: Config Testing Framework

In [ ]:
def test_write_config_valid():
    """Test valid WriteConfig."""
    config = WriteConfig(
        connection="delta",
        format="delta",
        table="sales",
        mode="overwrite"
    )
    assert config.connection == "delta"
    assert config.table == "sales"
    assert config.mode == WriteMode.OVERWRITE
    print("✅ test_write_config_valid passed")

def test_write_config_missing_table_and_path():
    """Test WriteConfig with neither table nor path fails."""
    try:
        config = WriteConfig(
            connection="local",
            format="csv"
            # Missing both table and path!
        )
        assert False, "Should have raised ValidationError"
    except ValidationError as e:
        assert "table" in str(e).lower() or "path" in str(e).lower()
        print("✅ test_write_config_missing_table_and_path passed")

def test_schedule_config_cron_valid():
    """Test valid cron schedule."""
    config = ScheduleConfig(
        type="cron",
        cron_expression="0 0 * * *"
    )
    assert config.type == ScheduleType.CRON
    assert config.cron_expression == "0 0 * * *"
    print("✅ test_schedule_config_cron_valid passed")

def test_schedule_config_cron_missing_expression():
    """Test cron schedule without expression fails."""
    try:
        config = ScheduleConfig(type="cron")
        assert False, "Should have raised ValidationError"
    except ValidationError as e:
        assert "cron_expression" in str(e).lower()
        print("✅ test_schedule_config_cron_missing_expression passed")

def test_schedule_config_invalid_cron():
    """Test invalid cron expression fails."""
    try:
        config = ScheduleConfig(
            type="cron",
            cron_expression="invalid"
        )
        assert False, "Should have raised ValidationError"
    except ValidationError:
        print("✅ test_schedule_config_invalid_cron passed")

def test_partition_config_requires_columns():
    """Test partitioning requires columns."""
    try:
        config = WriteConfig(
            connection="delta",
            format="delta",
            table="sales",
            partition_strategy="hash"
            # Missing partition_columns!
        )
        assert False, "Should have raised ValidationError"
    except ValidationError as e:
        assert "partition" in str(e).lower()
        print("✅ test_partition_config_requires_columns passed")

# Run all tests
print("Running config tests...\n")
test_write_config_valid()
test_write_config_missing_table_and_path()
test_schedule_config_cron_valid()
test_schedule_config_cron_missing_expression()
test_schedule_config_invalid_cron()
test_partition_config_requires_columns()
print("\n🎉 All tests passed!")

---

## 🎯 Key Takeaways

These solutions demonstrate:

1. **Enum Design**: Clear, type-safe constants
2. **Field Validators**: Validate individual fields with regex, constraints
3. **Model Validators**: Cross-field validation and business rules
4. **Discriminated Unions**: Polymorphic configs based on type
5. **Error Messages**: Clear, actionable, context-rich
6. **Config Merging**: Environment-specific overrides
7. **Testing**: Comprehensive validation of both success and failure cases

**Production Patterns**:
- Always validate early (load-time, not runtime)
- Provide helpful defaults
- Use `default_factory` for mutable defaults
- Write clear error messages with suggestions
- Test both valid and invalid configs
- Document fields with descriptions

These patterns are used throughout Odibi and in production data platforms!